<left>FINM 32000 - Numerical Methods</left>
<left>Spring 2023</left>
<br>
<h1><center> Homework 6 </center></h1>
<center>Due - 23:59 [CST] May 12th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

#### Imports

In [1]:
import numpy as np
from scipy.stats import norm

#### Helper Functions

In [2]:
class MultiGBM:
    
    def __init__(self,S0,r,correlations,sigma):
        self.S0 = S0
        self.r = r
        self.correlations = correlations
        self.sigma = sigma

In [3]:
class CallOnBasket:
    
    def __init__(self,K,T,weights):
        self.K = K
        self.T = T
        self.weights = weights

In [4]:
class MC:
    
    def __init__(self, M, antithetic, control, seed):
        self.M = M                                  # How many simulations 
        self.antithetic = antithetic
        self.control = control
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_callonbasket_multiGBM(self, contract,dynamics):
        
        # You complete the coding of this function.
        # self.rng.multivariate_normal may be useful.
        # See documentation for numpy.random.Generator.multivariate_normal
        # as self.rng is an instance of numpy.random.Generator
        
        # You are not required to support the case where MC.control = MC.antithetic = True
        # (simultaneous use of control variate and antithetic)
        # But you are required to support the other 3 possible settings of MC.antithetic/MC.control 
        # namely False/False, True/False, False/True.
        # (ordinary MC, antithetic without control, control without antithetic)
        drift = (dynamics.r - np.matmul(dynamics.sigma**2, contract.weights)) * contract.T
        W_T = self.rng.multivariate_normal(mean = np.array([0.0, 0.0]), cov = dynamics.correlations, size = self.M)
        stock_price = dynamics.S0 * np.exp(drift + np.sqrt(contract.T) * dynamics.sigma.dot(W_T.T).T)
        call_price = np.maximum(stock_price.dot(contract.weights) - contract.K, 0.0)

        if self.antithetic:
            stock_price_tilda = dynamics.S0 * np.exp(drift + np.sqrt(contract.T) * dynamics.sigma.dot(-W_T.T).T)
            call_price_tilda = np.maximum(stock_price_tilda.dot(contract.weights) - contract.K, 0.0)
            call_price = (call_price + call_price_tilda)/2
        elif self.control:
            call_price_star = np.maximum(np.sqrt(stock_price[:, 0] * stock_price[:, 1]) - contract.K, 0.0)
            # Black Scholes part
            S0_BS = np.sqrt(dynamics.S0[0] * dynamics.S0[1])
            sigma_BS = np.sqrt((dynamics.sigma[0, 0]**2 +
                                2*dynamics.correlations[0, 1]*dynamics.sigma[0, 0]*dynamics.sigma[1, 1] +
                                dynamics.sigma[1, 1]**2))
            d_1 = (np.log(S0_BS/contract.K) +
                   ((dynamics.r) + sigma_BS ** 2 / 2) * contract.T) / (sigma_BS * np.sqrt(contract.T))

            d_2 = d_1 - (sigma_BS * np.sqrt(contract.T))
            C_star = norm.cdf(d_1) * S0_BS - norm.cdf(d_2) * contract.K * np.exp(-dynamics.r * contract.T)
            # MLE beta
            beta_star = np.cov(call_price, call_price_star)[0, 1]/np.var(call_price_star)
            call_price = call_price + beta_star * (C_star - call_price_star)

        standard_error = np.std(call_price)/np.sqrt(self.M)
        call_price = np.mean(call_price)

        return(call_price, standard_error)

In [5]:
class GBM:

    def __init__(self, sigma, r, S0):
        self.sigma = sigma
        self.r = r
        self.S0 = S0

In [6]:
class CallOption:

    def __init__(self, K, T):
        self.K = K
        self.T = T

In [7]:
class MCimportance:

    def __init__(self, M, lamb, seed):
        self.M = M  # How many simulations
        self.lamb = lamb  # drift adjustment
        self.rng = np.random.default_rng(
            seed=seed)  # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_call_GBM(self, contract, dynamics):
        # You complete the coding of this function.
        # self.rng.normal may be useful.
        # See documentation for numpy.random.Generator.normal
        # as self.rng is an instance of numpy.random.Generator
        drift = (dynamics.r - dynamics.sigma**2/2 + self.lamb * dynamics.sigma) * contract.T
        BM = self.rng.normal(scale = np.sqrt(contract.T), size = self.M)
        W_T = BM - self.lamb * contract.T
        stock_price = dynamics.S0 * np.exp(drift + dynamics.sigma * W_T)
        call_price = np.maximum(stock_price - contract.K, 0.0)
        standard_error = np.std(call_price)/np.sqrt(self.M)
        call_price = np.mean(call_price)

        return (call_price, standard_error)

# Problem 1.

In [8]:
hw6p1dynamics = MultiGBM(S0=np.array([100, 110]), r=0.05,
                         correlations=np.array([[1, 0.8], [0.8, 1]]),  #You fill this in with a 2x2 correlation matrix
                         sigma=np.diag([0.3, 0.2]))

In [9]:
hw6p1contract = CallOnBasket(K=110, T=1.0, weights=np.array([1 / 2, 1 / 2]))

## (a)

The definition of $\mathbf{X}_t$ is:

$$
\mathbf{X}_t = \begin{pmatrix}
X^{[1]}_t \\ X^{[2]}_t
\end{pmatrix} =
\begin{pmatrix}
\log S^{[1]}_t \\ \log S^{[2]}_t
\end{pmatrix}
$$

Using Ito's formula III for each $X^{[j]}_t$,

$$
\begin{aligned}
dX^{[j]}_t &= \frac{1}{S^{[j]}_t} dS^{[j]}_t -
\frac{1}{2S^{[j]}^2_t} d<S^{[j]}>_t
\end{aligned}
$$

The given GBM dynamics is:

$$
dS^{[j]}_t = r S^{[j]}_t dt +
\sigma_{[j]} S^{[j]}_t dW^{[j]}_t
$$

The dynamics for $X^{[j]}_t$ becomes

$$
\begin{aligned}
dX^{[j]}_t &= \frac{1}{S^{[j]}_t}
\left(r S^{[j]}_t dt + \sigma_{[j]} S^{[j]}_t dW^{[j]}_t \right) -
\frac{1}{2S^{[j]}^2_t} (\sigma_{[j]} S^{[j]}_t)^2 dt \\
&= \left(r - \frac{\sigma_{[j]}^2}{2} \right) dt +
\sigma_{[j]} dW^{[j]}_t
\end{aligned}
$$

From this we may infer that:

$$
\begin{aligned}
\int_0^T 1 d X^{[j]}_t &= \int_0^T \left(r - \frac{\sigma_{[j]}^2}{2} \right) dt +
\int_0^T \sigma_{[j]} dW^{[j]}_t \\
\Leftrightarrow
X^{[j]}_T - X^{[j]}_0 &= \left(r - \frac{\sigma_{[j]}^2}{2} \right)T +
\sigma_{[j]} W^{[j]}_T \\
\Leftrightarrow
X^{[j]}_T &= X^{[j]}_0 + \left(r - \frac{\sigma_{[j]}^2}{2} \right)T +
\sigma_{[j]} W^{[j]}_T
\end{aligned}
$$

Therefore, if we let $\Sigma = \begin{bmatrix} \sigma_{[1]} & 0 \\ 0 & \sigma_{[2]} \end{bmatrix}$,
then $\mathbf{X}_T$ would become some constant plus $\Sigma \mathbf{W}_T$.

$$
\begin{aligned}
\mathbf{X}_T &= \begin{pmatrix}
X^{[1]}_0 + \left(r - \frac{\sigma_{[1]}^2}{2} \right)T +
\sigma_{[1]} W^{[1]}_T \\
X^{[2]}_0 + \left(r - \frac{\sigma_{[2]}^2}{2} \right)T +
\sigma_{[2]} W^{[2]}_T
\end{pmatrix} \\
&= \begin{pmatrix}
X^{[1]}_0 + \left(r - \frac{\sigma_{[1]}^2}{2} \right)T \\
X^{[2]}_0 + \left(r - \frac{\sigma_{[2]}^2}{2} \right)T
\end{pmatrix} +
\begin{pmatrix}
\sigma_{[1]} W^{[1]}_T \\
\sigma_{[2]} W^{[2]}_T
\end{pmatrix} \\
&=  \begin{pmatrix}
X^{[1]}_0 + \left(r - \frac{\sigma_{[1]}^2}{2} \right)T \\
X^{[2]}_0 + \left(r - \frac{\sigma_{[2]}^2}{2} \right)T
\end{pmatrix} +
\begin{bmatrix}
\sigma_{[1]} & 0 \\
0 & \sigma_{[2]}
\end{bmatrix}
\begin{pmatrix}
W^{[1]}_T \\
W^{[2]}_T
\end{pmatrix} \\
&= \begin{pmatrix}
X^{[1]}_0 + \left(r - \frac{\sigma_{[1]}^2}{2} \right)T \\
X^{[2]}_0 + \left(r - \frac{\sigma_{[2]}^2}{2} \right)T
\end{pmatrix} +
\Sigma
\mathbf{W}_T
\end{aligned}
$$

Moreover,

$$
\begin{aligned}
Cov(\mathbf{X}_T) &= \mathbf{E}(\mathbf{X}_T \mathbf{X}_T^T) \\
&= \mathbf{E}(\Sigma \mathbf{W}_T \mathbf{W}_T^T \Sigma^T) \\
&= \Sigma Cov(\mathbf{W}_T) \Sigma^T
\end{aligned}
$$

It was given that the correlation between $W^{[1]}$ and $W^{[2]}$ is $0.8$.

$$
\therefore
Cov(\mathbf{W}_T) = \begin{pmatrix}
T & 0.8 T \\ 0.8 T & T
\end{pmatrix}
$$

Lastly, substituting the values $\sigma_{[1]} = 0.3$ and $\sigma_{[2]} = 0.2$,

$$
\begin{aligned}
Cov(\mathbf{X}_T) &= \begin{pmatrix}
0.3 & 0 \\ 0 & 0.2
\end{pmatrix}
\begin{pmatrix}
T & 0.8 T \\ 0.8 T & T
\end{pmatrix}
\begin{pmatrix}
0.3 & 0 \\ 0 & 0.2
\end{pmatrix} \\
&= \begin{pmatrix}
0.09 T & 0.048 T \\ 0.048 T & 0.04 T
\end{pmatrix} \\
&= \begin{pmatrix}
0.09 & 0.048 \\ 0.048 & 0.04
\end{pmatrix} T
\end{aligned}
$$

## (b)

In [10]:
hw6p1bMC=MC(M=10000, antithetic=False, control=False, seed=0)
(call_price_ordinary, std_err_ordinary) = hw6p1bMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_ordinary, std_err_ordinary)

10.363539588654907 0.1766098446757389


## (c)

In [11]:
hw6p1cMC=MC(M=10000,antithetic=True,control=False,seed=0)
(call_price_AV, std_err_AV) = hw6p1cMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_AV, std_err_AV)

10.439551716001315 0.09962849057094109


## (d)

Fist of all,

$$
\log G_t = \frac{1}{2} \log S^{[1]}_t +
\frac{1}{2} \log S^{[2]}_t
$$

Therefore,

$$
\begin{aligned}
\mathbf{E}[\log G_t] &=
\mathbf{E}\left[\frac{1}{2} \log S^{[1]_t} + \frac{1}{2} \log S^{[2]_t} \right] \\
&= \frac{1}{2} \mathbf{E}[\log S^{[1]}_t] +
\frac{1}{2} \mathbf{E}[\log S^{[2]}_t] \\
&= \frac{1}{2} \mathbf{E}[X^{[1]}_t] +
\frac{1}{2} \mathbf{E}[X^{[2]}_t]
\end{aligned}
$$

We know from (a) that

$$
\mathbf{E}[X^{[j]}_t] =
X^{[j]}_0 + \left(r - \frac{\sigma_{[j]}^2}{2} \right)t
$$

Moreover,

$$
\begin{aligned}
\mathbf{E}[\log G_t] &=
\frac{1}{2} \mathbf{E}[X^{[1]}_t] +
\frac{1}{2} \mathbf{E}[X^{[2]}_t] \\
&= \frac{1}{2} \left(X^{[1]}_0 + \left(r - \frac{\sigma_{[1]}^2}{2} \right)t \right) +
\frac{1}{2} \left(X^{[2]}_0 + \left(r - \frac{\sigma_{[2]}^2}{2} \right)t \right) \\
&= \frac{1}{2} (X^{[1]}_0 + X^{[2]}_0) +
\left(r - \frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4} \right) t \\
&= \frac{1}{2} (\log S^{[1]}_0 + \log S^{[2]}_0) +
\left(r - \frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4} \right) t \\
&= \frac{1}{2} \log(S^{[1]}_0 S^{[2]}_0) +
\left(r - \frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4} \right) t
\end{aligned}
$$

$$
\therefore
\mathbf{E}[\log G_T] =
\frac{1}{2} \log(S^{[1]}_0 S^{[2]}_0) +
\left(r - \frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4} \right) T
$$

Now for the variance,

$$
\begin{aligned}
Var(\log G_t) &= Var\left(\frac{1}{2} \log S^{[1]}_t + \frac{1}{2} \log S^{[2]}_t \right) \\
&= \frac{1}{4} Var(\log S^{[1]}_t) +
\frac{1}{4} Var(\log S^{[2]}_t) +
\frac{1}{2} Cov(\log S^{[1]}_t, \log S^{[2]}_t) \\
&= \frac{1}{4} Var(X^{[1]}_t) +
\frac{1}{4} Var(X^{[2]}_t) +
\frac{1}{2} Cov(X^{[1]}_t, X^{[2]}_t)
\end{aligned}
$$

We also know from (a) that

$$
Var(X^{[j]}_t) = \sigma_{[j]}^2 t \\
Cov(X^{[1]}_t, X^{[2]}_t) = \rho \sigma_{[1]} \sigma_{[2]} t
$$

Therefore,

$$
\begin{aligned}
Var(\log G_t) &= \frac{1}{4} Var(X^{[1]}_t) +
\frac{1}{4} Var(X^{[2]}_t) +
\frac{1}{2} Cov(X^{[1]}_t, X^{[2]}_t) \\
&= \frac{\sigma_{[1]}^2 + 2 \rho \sigma_{[1]} \sigma_{[2]} + \sigma_{[2]}^2}{4} t
\end{aligned}
$$

Finally,

$$
Var(\log G_T) = \frac{\sigma_{[1]}^2 + 2 \rho \sigma_{[1]} \sigma_{[2]} + \sigma_{[2]}^2}{4} T
$$

## (e)

Given our findings in (d),

$$
C^G = C^{BS}
\left(\frac{1}{2} \log(S^{[1]}_0 S^{[2]}_0), 0, K, T, \left(r - \frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4} \right),
r, \frac{\sigma_{[1]}^2 + 2 \rho \sigma_{[1]} \sigma_{[2]} + \sigma_{[2]}^2}{4} \right)
$$

## (f)

In [12]:
hw6p1fMC=MC(M=10000,antithetic=False,control=True,seed=0)
(call_price_CV, std_err_CV) = hw6p1fMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_CV, std_err_CV)

20.128089649570253 0.0047029004699359065


## Problem 2

In [13]:
hw6p2dynamics=GBM(sigma=0.2,r=0.02,S0=100)

In [14]:
hw6p2contract=CallOption(K=150,T=1)

## (a)

In [15]:
hw6p2aMC=MCimportance(M=100000,lamb=0,seed=0) #zero drift adjustment gives ordinary MC

(call_price_ordinary, std_err_ordinary) =  hw6p2aMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_ordinary, std_err_ordinary)

0.25780827419770297 0.007762972399018178


## (b)

Under the new probability measure $\mathbf{P}^*$,

$$
W_t = W^*_t + \lambda t
$$

Now using Ito's II formula,

$$
\begin{aligned}
dW_t &= \lambda dt + dW^*_t + \frac{1}{2} d<W^*>_t \\
&= \lambda dt + dW^*_t
\end{aligned}
$$

Applying this to get the dynamics of $S_t$ in terms of the new probability measures gives:

$$
dS_t = \left(r + \sigma \lambda \right) S_t dt + \sigma S_t dW^*_t
$$

Now using the properties of GBM and the findings in 1 (a) we know that

$$
\log S_T = \log S_0 + \left(r + \sigma \lambda - \frac{\sigma^2}{2} \right)T + \sigma W^*_T
$$

Under the new probability measure $\mathbf{P}^*$,

$$
\mathbf{E}^*[\log S_T] = \log S_0 + \left(r + \sigma \lambda - \frac{\sigma^2}{2} \right)T
$$

since $W^*_T$ has $0$ drift in the new measure.

Therefore, the $\mathbf{P}^*$-expectation of $S_T$ is:

$$
\mathbf{E}^*[S_T] = S_0 e^{\left(r + \sigma \lambda - \frac{\sigma^2}{2} \right)T}
$$

Now, with the given values, the $\lambda$ that makes $\mathbf{E}^*[S_T] = 165$ is shown below

In [16]:
E_star_ST = 165
lambda_star = (np.log(E_star_ST/hw6p2dynamics.S0)/hw6p2contract.T
               - hw6p2dynamics.r + hw6p2dynamics.sigma**2/2)/hw6p2dynamics.sigma
print(lambda_star)

2.5038764395624455


## (c)

In [17]:
hw6p2cMC=MCimportance(M=100000,lamb = lambda_star,seed=0)
# Fill in the lamb parameter with the lambda that you compute in (b)
(call_price_importsamp, std_err_importsamp) =  hw6p2cMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_importsamp, std_err_importsamp)

0.2578082741977031 0.0077629723990181795
